In [2]:
#!pip install twilio,python-dotenv

In [1]:
import pandas as pd
import os

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

from twilio.rest import Client
import time


In [2]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="Config.env")

twilio_acc = os.getenv('TWILIO_ACC')
twilio_secret = os.getenv('TWILIO_SECRET')
twilio_phone = os.getenv('TWILIO_PHONE')
WEATHER_API = os.getenv('WEATHER_API')

# Api call 

In [12]:
query = 'Poza Rica'
url = "http://api.weatherapi.com/v1/forecast.json?key=" + WEATHER_API + "&q=" + query + "&days=1&aqi=no&alerts=no"

In [13]:
response = requests.get(url).json()

# Extract for all day

In [14]:
def get_forecast(response,i):
    general = response['forecast']['forecastday'][0]['hour'][i]
    fecha= general['time'].split()[0]
    hour = general['time'].split()[1].split(":")[0]
    condition = general["condition"]['text']
    temp_C = general["temp_c"]
    will_rain = general["will_it_rain"]
    chance_of_rain = general['chance_of_rain']
    
    return fecha,hour,condition,temp_C,will_rain,chance_of_rain
    

In [15]:
data = []

for x in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour="green"):
    data.append(get_forecast(response,x))
data

100%|██████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<?, ?it/s]


[('2023-08-09', '00', 'Overcast', 27.6, 0, 0),
 ('2023-08-09', '01', 'Overcast', 27.4, 0, 0),
 ('2023-08-09', '02', 'Cloudy', 27.2, 0, 0),
 ('2023-08-09', '03', 'Cloudy', 26.7, 0, 0),
 ('2023-08-09', '04', 'Fog', 24.0, 0, 0),
 ('2023-08-09', '05', 'Fog', 23.3, 0, 0),
 ('2023-08-09', '06', 'Fog', 23.1, 0, 0),
 ('2023-08-09', '07', 'Fog', 23.2, 0, 0),
 ('2023-08-09', '08', 'Cloudy', 24.4, 0, 0),
 ('2023-08-09', '09', 'Cloudy', 26.1, 0, 0),
 ('2023-08-09', '10', 'Cloudy', 27.7, 0, 0),
 ('2023-08-09', '11', 'Sunny', 34.0, 0, 0),
 ('2023-08-09', '12', 'Sunny', 36.8, 0, 0),
 ('2023-08-09', '13', 'Sunny', 38.9, 0, 0),
 ('2023-08-09', '14', 'Sunny', 40.1, 0, 0),
 ('2023-08-09', '15', 'Sunny', 40.6, 0, 0),
 ('2023-08-09', '16', 'Sunny', 39.9, 0, 0),
 ('2023-08-09', '17', 'Sunny', 37.8, 0, 0),
 ('2023-08-09', '18', 'Sunny', 35.4, 0, 0),
 ('2023-08-09', '19', 'Sunny', 32.8, 0, 0),
 ('2023-08-09', '20', 'Cloudy', 29.6, 0, 0),
 ('2023-08-09', '21', 'Overcast', 28.3, 0, 0),
 ('2023-08-09', '22', 'Cl

In [16]:
col = ["fecha","hour","condition","temp_C","will_rain","chance_of_rain"]
df = pd.DataFrame(data,columns=col)
df

,fecha,hour,condition,temp_C,will_rain,chance_of_rain
0,2023-08-09,00,Overcast,27.6,0,0
1,2023-08-09,01,Overcast,27.4,0,0
2,2023-08-09,02,Cloudy,27.2,0,0
3,2023-08-09,03,Cloudy,26.7,0,0
4,2023-08-09,04,Fog,24.0,0,0
5,2023-08-09,05,Fog,23.3,0,0
6,2023-08-09,06,Fog,23.1,0,0
7,2023-08-09,07,Fog,23.2,0,0
8,2023-08-09,08,Cloudy,24.4,0,0
9,2023-08-09,09,Cloudy,26.1,0,0


In [17]:
df_rain =  df #df[df["will_rain"] == 1]
df_rain = df_rain[["hour","condition"]]
df_rain.set_index('hour',inplace=True)

In [18]:
df_rain

,condition
hour,
00,Overcast
01,Overcast
02,Cloudy
03,Cloudy
04,Fog
05,Fog
06,Fog
07,Fog
08,Cloudy


# Message

In [19]:
message = "\n Hola el pronostico del tiempo de lluvia hoy " + df['fecha'][0] + ' en ' + query + ' es \n\n\n' + str(df_rain)

In [20]:
message

'\n Hola JC soy juanca, te mande este mensaje para que supieras el tiempo del dia:  2023-08-09 en Poza Rica es \n\n\n     condition\nhour          \n00    Overcast\n01    Overcast\n02      Cloudy\n03      Cloudy\n04         Fog\n05         Fog\n06         Fog\n07         Fog\n08      Cloudy\n09      Cloudy\n10      Cloudy\n11       Sunny\n12       Sunny\n13       Sunny\n14       Sunny\n15       Sunny\n16       Sunny\n17       Sunny\n18       Sunny\n19       Sunny\n20      Cloudy\n21    Overcast\n22       Clear\n23       Clear'

In [21]:
client = Client(twilio_acc, twilio_secret)

message = client.messages \
    .create(
         body=message,
         from_=twilio_phone,
         to='+528120075281'
     )
print(message.sid)


TwilioRestException: HTTP 400 error: Unable to create record: The number  is unverified. Trial accounts cannot send messages to unverified numbers; verify  at twilio.com/user/account/phone-numbers/verified, or purchase a Twilio number to send messages to unverified numbers.